In [1]:
# 3.2. Linear Regression Implementation from Scratch¶
%matplotlib
import matplotlib.pyplot as plt
import random
import torch
from d2l import torch as d2l
import os
os.environ["KMP_DUPLICATE_LIB_OK"]  =  "TRUE"

Using matplotlib backend: Qt5Agg


In [2]:
# 3.2.1. Generating the Dataset
import torch
def synthetic_data(w, b, num_examples):  #@save
    """Generate y = Xw + b + noise."""
    X = torch.normal(0, 1, (num_examples, len(w)))
    # torch.normal(means, std, out=None)
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [3]:
print('features:', features[0], '\nlabel:', labels[0])

features: tensor([-1.2199,  1.3261]) 
label: tensor([-2.7416])


In [4]:
d2l.set_figsize()
# The semicolon is for displaying the plot only
d2l.plt.scatter(features[:, (1)].detach().numpy(),
                labels.detach().numpy(), 1);

In [5]:
# 3.2.2 Reading the Dataset
def data_iter(batch_size, features, labels):
    """shuffle the dataset and access it in minibatches."""
    num_examples = len(features)
    # print(num_examples) : 1000
    indices = list(range(num_examples))
    # print(indices) : [0, 1, 2, ..., 999]
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    # random.shuffle(list) randomly sorts elements of list
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i:min(i +
                                                   batch_size, num_examples)])
        print(batch_indices)
        yield features[batch_indices], labels[batch_indices]

In [6]:
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

tensor([914, 101, 399, 568, 926, 654, 168, 406,  39, 839])
tensor([[-1.1949,  0.4809],
        [ 0.3609, -0.7046],
        [-0.5775, -1.3774],
        [ 0.0780,  0.3229],
        [ 0.3615, -0.6590],
        [ 1.7727, -0.3735],
        [ 0.5623, -1.2538],
        [ 1.5614, -0.8310],
        [-0.7144, -1.7469],
        [ 0.4651, -0.9299]]) 
 tensor([[ 0.1783],
        [ 7.3355],
        [ 7.7399],
        [ 3.2538],
        [ 7.1602],
        [ 9.0244],
        [ 9.5817],
        [10.1529],
        [ 8.7154],
        [ 8.2854]])


In [7]:
# 3.2.3. Initializing Model Parameters
w = torch.normal(0, 0.01, size=(2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [8]:
# 3.2.4. Defining the Model
def linreg(X, w, b):  #@save
    """The linear regression model."""
    return torch.matmul(X, w) + b

In [9]:
# 3.2.5. Defining the Loss Function
def squared_loss(y_hat, y):  #@save
    """Squared loss."""
    return (y_hat - y.reshape(y_hat.shape))**2 / 2

In [10]:
# 3.2.6. Defining the Optimization Algorithm
def sgd(params, lr, batch_size):  #@save
    # lr => learning rate
    """Minibatch stochastic gradient descent."""
    # when we're evaluating, in other words, there's no need to find gradients, we can wrap the inference code inside with torch.no_grad():
    # by doing so, we could reduce the possible calculation and RAM consumption
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

In [11]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)  # Minibatch loss in `X` and `y`
        # Compute gradient on `l` with respect to [`w`, `b`]
        l.sum().backward()
        sgd([w, b], lr, batch_size)  # Update parameters using their gradient
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

tensor([685, 962, 247, 549, 966, 460, 686, 192, 329, 717])
tensor([522, 861, 890, 882, 211, 767, 573, 168, 809,  84])
tensor([407,  74, 843, 337, 844, 122, 385, 718, 946, 421])
tensor([860, 342, 971, 166, 110, 111, 402, 442, 411, 284])
tensor([ 61, 596, 533, 668,  46, 438, 126, 400, 352, 874])
tensor([733, 488, 687, 569,  39, 418,  34, 943, 234, 434])
tensor([102, 704, 979, 847, 156, 991, 292, 673, 676, 155])
tensor([806, 653, 572, 133, 708,  62, 316, 702, 738, 212])
tensor([205,  21,  23, 249, 136, 288, 476, 926, 351, 951])
tensor([ 73, 808, 101, 313, 380, 157, 524, 375, 617, 870])
tensor([580, 897,  99, 603, 149, 472, 426, 815,  57, 456])
tensor([465, 363, 608, 204, 917, 880, 320,  41, 237, 508])
tensor([185, 933, 183, 410, 467, 217, 127, 609, 825, 945])
tensor([696, 794, 830, 992,  16, 873, 832, 646, 674, 399])
tensor([ 43, 797, 436,  32, 409, 682, 174, 405, 441, 490])
tensor([671, 647, 251, 563, 119, 145, 699, 152, 833, 578])
tensor([679, 997, 900,   3, 216, 555, 457, 826, 286, 530

tensor([293, 440, 811, 768, 520, 903, 390, 174, 420, 186])
tensor([786, 970, 974, 489, 906, 785, 457, 119, 400, 356])
tensor([530, 208, 128, 648, 509, 792, 313, 746, 125, 980])
tensor([  5, 816, 288, 898, 801,  49, 368, 370,  19, 789])
tensor([483, 459, 482, 129, 893, 405, 609, 888, 881, 229])
tensor([576, 626, 953, 851, 581, 763, 371, 410, 511, 296])
tensor([159, 413, 194, 310, 552, 393, 338,  46, 538, 286])
tensor([216, 287, 628, 775, 845,  55, 870, 399,  11, 308])
tensor([452, 150, 949, 610, 444, 432, 395, 919, 976, 675])
tensor([941,  95, 592, 691, 921, 431, 197, 702, 320, 512])
tensor([126, 124, 146, 506, 993, 451,  53, 708,  44, 391])
tensor([947, 295,  90, 569,  72,  69, 955, 127, 957, 905])
tensor([550, 884, 226,  45, 235, 141, 975,  27, 780, 524])
tensor([342, 224,  51, 442, 156, 694, 133, 853, 807, 551])
tensor([168, 134, 990, 578, 833, 302, 396, 522, 719, 570])
tensor([707, 435, 411, 157, 213, 729,  91, 455,  34, 203])
tensor([596, 173, 494, 248, 153, 158,  77, 891, 855, 201

tensor([564, 289, 539, 860, 770,  64, 837, 510, 547, 381])
tensor([530, 653, 220,  61, 137, 133, 615, 238, 495, 847])
tensor([626, 260, 301, 961, 949, 855,  85,  97, 300,  92])
tensor([278, 757,  78, 515, 863, 583, 686, 102, 361, 607])
tensor([882,  50, 210, 908, 395, 486, 771, 279, 703, 148])
tensor([945, 706, 144,  62, 551, 815, 888,  87, 431, 291])
tensor([582, 576, 466,  69, 593, 683, 392, 141, 367, 717])
tensor([804, 741, 317,  72,  60,  84, 463, 232, 324, 302])
tensor([614, 244, 932, 363, 866, 458,  93, 345, 224, 424])
tensor([839, 171, 357, 813, 468, 353, 387, 975, 707, 581])
tensor([783, 223, 262, 565, 106, 199, 248, 314, 418, 413])
tensor([341, 253, 996, 109, 997, 524, 518, 280, 931, 299])
tensor([972, 180, 196, 411, 787,  39, 695, 175, 139, 385])
tensor([119, 658, 349, 661, 493, 494, 739,  80, 451, 665])
tensor([ 11, 379, 342, 268, 740, 528, 756,  35, 233,  54])
epoch 3, loss 0.000050


In [12]:
print(f'error in estimating w: {true_w - w.reshape(true_w.shape)}')
print(f'error in estimating b: {true_b - b}')

error in estimating w: tensor([ 1.3399e-04, -5.5313e-05], grad_fn=<SubBackward0>)
error in estimating b: tensor([0.0003], grad_fn=<RsubBackward1>)
